In [21]:
import re

from base_gen_deletion_script import *

In [88]:
def run_command(command: List[str]) -> dict:
    """Run a shell command and return the output."""
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error running command: {' '.join(command)}")
        print(result.stderr)
        # sys.exit(1)
    ret = {
        "stdout": result.stdout,
        "stderr": result.stderr,
        "returncode": result.returncode
    }
    return ret 

def clean_text(input_text: str) -> str:
    """
    Cleans the input text by removing ANSI escape sequences and formatting codes.

    Args:
        input_text (str): Text containing ANSI escape sequences.

    Returns:
        str: Cleaned text without escape sequences.
    """
    # Regex pattern to match ANSI escape sequences
    ansi_escape_pattern = re.compile(r'\x1b\[[0-9;]*m')
    cleaned_text = ansi_escape_pattern.sub('', input_text)
    return cleaned_text

def get_gcloud_help_block(help_text: str, block_name: str, 
                          cmd_indent: int = 5, desc_indent: int = 8,
                          verbose: bool = False) -> dict:
    """
    Extracts a specific block of text from the help text based on the block name.

    Args:
        help_text (str): The entire help text from the gcloud command.
        block_name (str): The name of the block to extract.

    Returns:
        str: The block of text from the help text.
    """
    args_desc = {}

    in_block = False
    for line in clean_text(help_text).split("\n"):
        # print(line)

        # if the line starts with the block name
        if line.startswith(block_name): 
            in_block = True
        elif in_block and line.isupper(): 
            # check stop condition, when the line starts with all capital letters
            in_block = False
            break
        
        # print if in block and not new line
        if in_block and line != "": 
            if verbose: print(line)
            if line.startswith(" "*cmd_indent) and line[cmd_indent] != " ":
                tmp_arg = line.strip()
                args_desc[tmp_arg] = None
            elif line.startswith(" "*desc_indent):
                # remove front and back spaces
                args_desc[tmp_arg] = line.strip()
                tmp_arg = None
    return args_desc

In [90]:
out_dict = run_command(["gcloud", "compute", "networks", "--help"])

block = "GROUP"
get_gcloud_help_block(out_dict["stdout"], block)

{'peerings': 'List, create, and delete, and update VPC Network Peering.',
 'subnets': 'List, describe, and delete, and update Compute Engine subnetworks.',
 'vpc-access': 'Manage VPC Access Service resources.'}

In [93]:
out_dict = run_command(["gcloud", "compute", "networks", "peerings", "--help"])
print(clean_text(out_dict["stdout"]))

NAME
    gcloud compute networks peerings - list, create, and delete, and update VPC
        Network Peering

SYNOPSIS
    gcloud compute networks peerings COMMAND [GCLOUD_WIDE_FLAG ...]

DESCRIPTION
    Read and manipulate VPC Network Peering connections.

    For more information about VPC Network Peering, see the VPC Network Peering
    documentation (https://cloud.google.com/vpc/docs/vpc-peering).

    See also: Network API
    (https://cloud.google.com/compute/docs/reference/rest/v1/networks).

GCLOUD WIDE FLAGS
    These flags are available to all commands: --help.

    Run $ gcloud help for details.

COMMANDS
    COMMAND is one of the following:

     create
        Create a Compute Engine network peering.

     delete
        Delete a Compute Engine network peering.

     list
        List Google Compute Engine peerings.

     list-routes
        List received or advertised routes for a VPC network peering.

     update
        Update a Compute Engine network peering.



In [96]:
out_dict = run_command(["gcloud", "compute", "networks", "subnets", "--help"])
print(clean_text(out_dict["stdout"]))

NAME
    gcloud compute networks subnets - list, describe, and delete, and update
        Compute Engine subnetworks

SYNOPSIS
    gcloud compute networks subnets COMMAND [GCLOUD_WIDE_FLAG ...]

DESCRIPTION
    Read and manipulate VPC subnetworks.

    For more information about VPC, see the VPC documentation
    (https://cloud.google.com/vpc/docs/vpc).

    See also: Subnetworks API
    (https://cloud.google.com/compute/docs/reference/rest/v1/subnetworks).

GCLOUD WIDE FLAGS
    These flags are available to all commands: --help.

    Run $ gcloud help for details.

COMMANDS
    COMMAND is one of the following:

     add-iam-policy-binding
        Add an IAM policy binding to a Compute Engine subnetwork.

     create
        Define a subnet for a network in custom subnet mode.

     delete
        Delete Google Cloud subnetworks.

     describe
        Describe a Compute Engine subnetwork.

     expand-ip-range
        Expand the IP range of a Compute Engine subnetwork.

     get-iam-p

In [101]:
out_dict = run_command(["gcloud", "compute", "networks", "subnets", "list", "--uri"])
print(clean_text(out_dict["stdout"]))

https://www.googleapis.com/compute/v1/projects/terraform-437400/regions/us-central1/subnetworks/my-subnetwork



In [102]:
out_dict = run_command(["gcloud", "compute", "networks", "subnets", "list", "--help"])
print(clean_text(out_dict["stdout"]))

NAME
    gcloud compute networks subnets list - list Google Compute Engine
        subnetworks

SYNOPSIS
    gcloud compute networks subnets list [NAME ...] [--network=NETWORK]
        [--regexp=REGEXP, -r REGEXP] [--regions=REGION,[REGION,...]]
        [--filter=EXPRESSION] [--limit=LIMIT] [--page-size=PAGE_SIZE]
        [--sort-by=[FIELD,...]] [--uri] [GCLOUD_WIDE_FLAG ...]

DESCRIPTION
    gcloud compute networks subnets list displays all Google Compute Engine
    subnetworks in a project.

    By default, subnetworks from all regions are listed. The results can be
    narrowed down using a filter: --filter="region:( REGION ... )".

EXAMPLES
    To list all subnetworks in a project in table form, run:

        $ gcloud compute networks subnets list

    To list the URIs of all subnetworks in a project, run:

        $ gcloud compute networks subnets list --uri

    To list all subnetworks in the us-central1 and europe-west1 regions, run:

        $ gcloud compute networks subnets li

In [100]:
out_dict = run_command(["gcloud", "compute", "networks", "list", "--uri"])
print(clean_text(out_dict["stdout"]))

https://www.googleapis.com/compute/v1/projects/terraform-437400/global/networks/my-network

